In [9]:
from client import NineGagAPIClient

In [10]:
api = NineGagAPIClient()

In [11]:
api._get_json(url="")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
import requests

url = "https://9gag.com/v1/feed-posts/type/home"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36",
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Referer": "https://9gag.com/",
    "Accept-Language": "en-US,en;q=0.9",
    "Cookie": "____ri=6854; ____lo=DE; sign_up_referer=https%3A%2F%2F9gag.com%2F; ts1=9f9ecd24c7dd0e753be3d450d0c381f6ae9fab00; OptanonConsent=isGpcEnabled=0&datestamp=Tue+Jan+07+2025+16%3A04%3A54+GMT%2B0100+(Central+European+Standard+Time)&version=202408.1.0&browserGpcFlag=0&isIABGlobal=false&hosts=&consentId=d6bd00d0-d975-41a3-8a52-149f70856088&interactionCount=0&isAnonUser=1&landingPath=NotLandingPage&groups=C0004%3A1%2CC0002%3A1%2CC0003%3A1%2CC0001%3A1&AwaitingReconsent=false&geolocation=%3B; OptanonAlertBoxClosed=2025-01-07T15:04:54.894Z"
}
response = requests.get(url, headers=headers)
if response.status_code != 200:
    print("Failed to fetch data:", response.status_code)

#print(response.json())  # Inspect the structure

Failed to fetch data: 403


In [12]:
response

<Response [403]>